# Retrieval

In [9]:
from dotenv import load_dotenv
load_dotenv()

True

### Document Loader

In [ ]:
# 다양한 데이터소스에서 로드해오는 녀석
# WebBaseLoader: 웹페이지의 내용 읽어오기 (크롤링)
from langchain_community.document_loaders import WebBaseLoader  # 웹페이지를 크롤링해서 그 document를 로드해줌 

url = "https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8"

loader = WebBaseLoader(url)
documents = loader.load()

print(documents[0].page_content[:1000])





위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전



























본문으로 이동







주 메뉴





주 메뉴
사이드바로 이동
숨기기



		둘러보기
	


대문최근 바뀜요즘 화제임의의 문서로특수 문서 목록





		사용자 모임
	


사랑방사용자 모임관리 요청





		편집 안내
	


소개도움말정책과 지침질문방



















검색











검색






















보이기
















기부

계정 만들기

로그인








개인 도구





기부 계정 만들기 로그인





		로그아웃한 편집자를 위한 문서 더 알아보기



기여토론




























목차
사이드바로 이동
숨기기




처음 위치





1
최상위 정책








2
'정책과 지침'이란?








3
준수








4
집행








5
문서 내용








6
정책과 지침은 백과사전의 일부가 아닙니다








7
채택 과정




채택 과정 하위섹션 토글하기





7.1
제안과 채택








7.2
내용 변경






7.2.1
실질적인 변경










7.3
격하










8
같이 보기








9
외부 링크


















목차 토글







위키백과:정책과 지침



112개 언어




Afrikaansአማርኛअंगिकाالعربيةالدارجةمصرىঅসমীয়াАварAzərbaycancaتۆرکجهБашҡортсаBoarischБеларускаяБеларуская (тарашкевіца)БългарскиभोजपुरीBanjarবাংলাBrezhonegBosanskiCatalàНохчийнکوردیČeštinaCymraegDanskDeutschΕλληνικάEnglishEsperantoEspañolEuskaraفارسیSuomiFran

In [ ]:
# !pip install pypdf

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('./data/The_Adventures_of_Tom_Sawyer.pdf')
documents = loader.load()
print(len(documents))
print(documents[0].metadata)
print('=' * 100)
print(documents[2].page_content)

35
{'producer': '3-Heights(TM) PDF Optimization Shell 5.9.1.5 (http://www.pdf-tools.com)', 'creator': 'Acrobat PDFMaker 7.0 dla programu Word', 'creationdate': '2006-08-26T00:50:00+02:00', 'author': 'GOLDEN', 'company': 'c', 'title': 'Microsoft Word - 1', 'moddate': '2021-01-27T15:00:11+01:00', 'source': './data/The_Adventures_of_Tom_Sawyer.pdf', 'total_pages': 35, 'page': 0, 'page_label': '1'}
The Adventures of                 
Tom Sawyer 
 
MARK TWAIN 
Level 1 
 
Retold by Jacqueline Kehl                                                    
Series Editors: Andy Hopkins and Jocelyn Potter


### Embedding Model

In [ ]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

text = "The quick brown fox jumps over the lazt dog"
vector = embedding_model.embed_query(text)  # embed_query : 단일 문장을 임베딩할 때 사용용

print(len(vector))  # 한 페이지의 내용이 1536차원으로 임베딩됨 

1536


In [12]:
docs = [document.page_content for document in documents]
vects = embedding_model.embed_documents(docs)

print(len(vects))
print(len(vects[0]))

35
1536


### Vector Store

In [ ]:
from langchain.vectorstores import FAISS

txt_embeddings = list(zip(docs, vects))
# vector_store = FAISS.from_embeddings(txt_embeddings, embedding_model)   # 임베딩된 데이터를 불러와서 넣어주게 되면 from_embeddings
vector_store = FAISS.from_documents(documents, embedding_model)         # 원본문서만 있다면 from_documents / 결론적으로 똑같이 동작을 한다. (임베딩된 데이터가 있냐 없냐에 따라 선택적으로 사용하면 됨)
# 반환해주는 대상은 같지만 아래 셀에서 출력되는 내용은 차이가 있다.

In [16]:
vector_store.similarity_search("Tom Sawyer", k=3)   # "Tom Sawyer" : 질의 (키워드) / 벡터화 시킨 것으로 계산해야하니까 알아서 임베딩처리를 한 후에 유사도 검색을 해주게 됨 / k: 유사도가 높은 k개 

[Document(id='1b36cd1b-f800-45cb-86a8-0f32ab70fa76', metadata={'producer': '3-Heights(TM) PDF Optimization Shell 5.9.1.5 (http://www.pdf-tools.com)', 'creator': 'Acrobat PDFMaker 7.0 dla programu Word', 'creationdate': '2006-08-26T00:50:00+02:00', 'author': 'GOLDEN', 'company': 'c', 'title': 'Microsoft Word - 1', 'moddate': '2021-01-27T15:00:11+01:00', 'source': './data/The_Adventures_of_Tom_Sawyer.pdf', 'total_pages': 35, 'page': 4, 'page_label': '5'}, page_content='Introduction \n \n \nOne Saturday afternoon Tom wanted to have an adventure                    \nbecause he didn’t want to think about Injun Joe. He went \nto Huck and said, “I’m going to look for treasure. Do you \nwant to come with me?” \n \nTom Sawyer loves adventures. He has a lot of adventures \nat home, at school, and with his friends. He has one \nadventure in a cave. But why is he there? What does he \nsee in the cave? And why is he afraid? \n \nMark Twain (1835-1910) is a famous American writer. \nHis name was Sam

### Retriever

In [ ]:
# "질문과 관련된 정보를 찾아오는 역할을 하는 객체(또는 함수) / Retriever = 질문과 관련된 정보를 내부 저장소에서 찾아주는 도구

# 사용자가 질문하면 →

# Retriever가 내부 문서(혹은 벡터 DB, PDF, 노션 등) 중에서 관련 있는 문서들을 골라줌

# 그 문서들과 함께 질문을 모델에게 전달

# 모델이 더 정확하게 답변 가능!

In [ ]:
# 데이터베이스를 검색해서 llm에 참고를 할 수 있게 

# 벡터DB에서 유사한 것을 찾아서 입력된 질의에 대해서 응답할만한 참고자료를 찾는 역할을 한다고 보면 됨 

In [17]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0
)

retriever = vector_store.as_retriever()
print(retriever)

retrieval_qa = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type='stuff'
)

tags=['FAISS', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000028C74020DC0> search_kwargs={}


In [18]:
response = retrieval_qa.invoke('톰소여는 어떤 사람인가요?')
response

{'query': '톰소여는 어떤 사람인가요?',
 'result': '톰 소여는 모험을 사랑하는 소년으로, 집, 학교, 친구인 헉 핀과 조 하퍼와 함께 다양한 모험을 경험합니다. 그는 종종 장난을 치고, 친구를 도우려는 마음도 가지고 있으며, 특히 친구인 머프 포터를 도와주고 싶어하는 모습을 보입니다. 톰은 용감하고, 때로는 두려움을 느끼기도 하지만, 친구를 위해 행동하는 성격을 가지고 있습니다.'}